<h1 align="center">Check Questions</h1> 

Ответе на вопросы своими словами, если не выходит, то вернитесь к лекции дополнительным материалам:

**Вопрос 1**: В каком пространстве градиентный бустинг совершает градиентный спуск? Какова размерность этого пространства?

В пространстве меток объектов, размерность- количество меток объектов

**Вопрос 2**: Почему бустинг над глубокими деревьями это плохая идея?

Существует большая вероятность переобучения

**Вопрос 3**: Что предсказывает каждое дерево (что является признаками а что целевой переменной?)

Признаки - параметры объектов, целевая переменная - класс

<h1 align="center">Binary Boosting Implementation</h1> 

Давайте вспомним бустинг

#### Градиентный спуск

Самый простой метод минимизации функции, для оптимизации в каждый момент времени двигаемся по антиградиенту функции с каким-то шагом. 

$$w_{n+1} = w_n - s \cdot \frac{\partial f}{\partial w}$$

#### Градиентный бустинг

Теперь давайте представим, что на каждом шаге мы оптимизируем не параметры алгоритма $w$, а ответы нашего алгоритма $\hat{y}$.

**Обучение**: На каждом шаге, давайте предсказывать градиент на каждом объекте и "двигать" ответ в сторону улучшения (антиградиента).

**Алгоритм**:
- Первый алгоритм отвечает константу 
- Добавляем базовые алгоритмы $b_i$, $i = 1, .., N$:
    - $\hat{y} = \sum_{j=0}^{i-1} a_j b_j(x)$
    - Вычисляем градиент функции потерь ПО ОТВЕТАМ модели $g_{i-1} = \frac{\partial L(\hat{y},~~y)}{\partial \hat{y}}$ на каждом объекте  
    - Обучаем $b_i$ предсказывать текущий $g_{i-1}$ (Тут дерево не глубокое регрессионное дерево)
    - Дополняем композицию $\sum_{j=0}^{i-1} a_j b_j (x) + lr * b_i(x)$
    
    
#### Нужно реализовать двух классовый бустинг с логистической функцией потерь.     

**Функция потерь**:
Я вот думаю, что всем интересно какую-же функцию потерь выбрать $\mathcal{L}(\hat{y},y)=\log\left( 1 + e^{-\hat{y}y} \right)$

тут важный момент есть, даже не один
- $\hat{y}$ -- это ответ композиции, тоесть сумма ответов всех предыдущих деревьев
- Это скалярная функция и производная халява, но вот тут мы вам посчитали $$\frac{\partial \mathcal{L}}{\partial \hat{y}} = \frac{1}{1 + e^{-y\hat{y}}} \cdot (-ye^{-y\hat{y}})=-y\frac{1}{1 + e^{y\hat{y}}}$$

In [1]:
%pylab inline

import warnings
import numpy as np
import pandas as pd
from copy import deepcopy
from utils import plot_surface
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import cross_val_score
from sklearn.base import ClassifierMixin, BaseEstimator
from sklearn.ensemble import GradientBoostingClassifier
import math
import scipy
from sklearn import tree

warnings.simplefilter("ignore")

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [21]:
class BinaryBoostingClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_estimators, lr=0.1):
        self.lr = lr
        self.n_estimators = n_estimators

    def loss_grad(self, original_y, pred_y):
        #print(np.dot( np.array(original_y), np.array(pred_y) ))
        return np.array(original_y, )*(-scipy.special.expit( np.dot( - np.array(original_y), np.array(pred_y) )))
        #return np.array(original_y, ) * (-1) / float( 1 + math.exp( np.dot( np.array(original_y), np.array(pred_y) )) )
    

    def fit(self, X, original_y):
        clf = tree.DecisionTreeRegressor()
        clf = clf.fit(X, original_y)
        self.estimators_ = [clf]

        for i in range(self.n_estimators):
            grad = self.loss_grad(original_y, self._predict(X))

            estimator = tree.DecisionTreeRegressor()
            estimator = estimator.fit(X, grad)
            self.estimators_.append(estimator)

        return self

    def _predict(self, X):

        y_pred = self.estimators_[0].predict(X)
        for i in range(1, len(self.estimators_)):
            y_pred = y_pred  - self.lr * self.estimators_[i].predict(X)
        return y_pred

    def predict(self, X):

        y_pred_not_norm = self._predict(X)
        y_pred = [0] * len(y_pred_not_norm)
        for i in range(len(y_pred_not_norm)):
            if y_pred_not_norm[i] >= 0:
                y_pred[i] = 1
            else:
                y_pred[i] = -1
        return y_pred

<h1 align="center">Simple test</h1> 

In [7]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=500, n_features=2,
                           n_informative=2, n_redundant=0, n_repeated=0,
                           n_classes=2, n_clusters_per_class=2,
                           flip_y=0.05, class_sep=0.8, random_state=241)
y = 2*(y-0.5)

In [22]:
import sklearn
clf = BinaryBoostingClassifier(n_estimators=150).fit(X[0:30000], y[0:30000])
print(sklearn.metrics.accuracy_score(y[30001:32000], clf.predict(X[30001:32000]) ))
clf = sklearn.ensemble.GradientBoostingClassifier()
clf.fit(X[0:30000], y[0:30000])
y_pred = clf.predict(X[30001:32000])
print(sklearn.metrics.accuracy_score(y[30001:32000], y_pred ))

0.814407203602
0.867433716858


<h1 align="center">Adult test</h1> 

#### Скачайте https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data

In [9]:
adult = pd.read_csv(
    'adult.data', 
    names=[
        "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
        "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "cdCapital Loss",
        "Hours per week", "Country", "Target"], 
    header=None, na_values="?")
adult = pd.get_dummies(adult)
adult["Target"] = adult["Target_ >50K"]

In [10]:
X, y = adult[adult.columns[:-3]].values, adult[adult.columns[-1]].values
y = 2*(y-0.5)

In [39]:
<Сверьте качество своего алгоритма с GradientBoostingClassifier, оно должно быть примерно таким-же>

SyntaxError: invalid syntax (<ipython-input-39-08390be21502>, line 1)

In [20]:
import sklearn
clf = sklearn.ensemble.GradientBoostingClassifier()
clf.fit(X[0:500], y[0:500])
y_pred = clf.predict(X[20001:32000])
print(sklearn.metrics.accuracy_score(y[20001:32000], y_pred ))
clf = BinaryBoostingClassifier(n_estimators=100).fit(X[0:500], y[0:500])
print(sklearn.metrics.accuracy_score(y[20001:32000], clf.predict(X[20001:32000]) ))

0.8367363947
0.808984082007


<h1 align="center">Сдача ДЗ</h1> 

Заполните форму https://goo.gl/forms/sPE6gpRDNTOXQai12 
    - Качество вашего алгоритма на adult, один знак после запятой, без округления (0.86 -> 0.8 и тд) точность
    - BinaryBoostingClassifier.loss_grad ([-1, 1, 1], [-1, 1, -1]).sum()

In [12]:
BinaryBoostingClassifier(n_estimators=100).loss_grad ([-1, 1, 1], [-1, 1, -1]).sum()

-0.2689414213699951

In [17]:
X

array([[    39,  77516,     13, ...,      1,      0,      0],
       [    50,  83311,     13, ...,      1,      0,      0],
       [    38, 215646,      9, ...,      1,      0,      0],
       ..., 
       [    58, 151910,      9, ...,      1,      0,      0],
       [    22, 201490,      9, ...,      1,      0,      0],
       [    52, 287927,      9, ...,      1,      0,      0]])